First, we initialize the matroid and obtain the lattice of flats. We relabel the flats for simple presentation.

In [188]:
r, n = 5, 6

R.<q> = PolynomialRing(QQ)
matroid = matroids.Uniform(r, n)
flats = list(matroid.lattice_of_flats())

In [189]:
#plottting the geometric lattice:
#size = 0 for now

labels = {element: idx for idx, element in enumerate(flats)}
matroid.lattice_of_flats().plot(
    element_labels = labels, element_color = "white",
    figsize= 0, cover_color = "grey")

/home/nnutannep/miniforge3/envs/sage/lib/python3.11/site-packages/sage/repl/rich_output/display_manager.py:596: RichReprWarning: Exception in _rich_repr_ while displaying object: figsize should be positive, not 0.0
  warnings.warn(


Graphics object consisting of 230 graphics primitives

In [190]:
def invKL(M):
	R = PolynomialRing(QQ, 'q')
	q = R.gen(0)
	if M.rank() == 0:
		return R(1)
	L = M.lattice_of_flats()
	invKLpoly = 0
	for i in range(1,len(L)):
		Mcon = M.contract(L[i])
		Mres = M.delete(M.groundset()-L[i])
		Lres = Mres.lattice_of_flats()
		invKLpoly += (-1)^(Mres.rank())*Lres.kazhdan_lusztig_polynomial()*invKL(Mcon)
	return -invKLpoly

Guess 1:
$$Q_M(t) = Q_{M\setminus e}(t)-tQ_{M/e}(t)+\sum_{F\in S}{\tau(M/{F\cup e})t^{\text{crk}(F)/2}Q_{M\mid F}(t)}.$$

Functions needed for the r.h.s. will be written below:

In [191]:
#the right hand side of the equality

def q_tau(matroid, S, e):
    q_tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            q_tau.append(0)
        else:
            q_tau.append(invKL(modded_matroid).leading_coefficient())
    return q_tau

def tau(matroid, S, e):
    tau = [] 
    for F in S:
        modded_matroid = matroid.contract(F.union(e))
        if modded_matroid.rank() % 2 == 0:
            tau.append(0)
        else:
            tau.append(modded_matroid.lattice_of_flats().kazhdan_lusztig_polynomial().leading_coefficient())
    return tau

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

def delform_rhs(matroid, tau_func, inv_kl):
    e = matroid.flats(1)[0]
    flats = list(matroid.lattice_of_flats())

    deletion = matroid.delete(e).simplify()
    contraction = matroid.contract(e).simplify()

    set_S = list(the_set_S(flats, e))

    # Compute bv_tau_function and zip with set_S
    tau_values = tau_func(matroid, set_S, e)

    # Compute the summation over S
    sum_S = sum(
        x * q ** ((matroid.rank() - matroid.rank(y))//2) * inv_kl(matroid.delete(matroid.groundset() - y).simplify())
        for x, y in zip(tau_values, set_S)
    )
    # Compute the RHS
    return inv_kl(deletion) - q * inv_kl(contraction) + sum_S

In [192]:
print(invKL(matroid))

delform_rhs(matroid, q_tau, invKL)

5*q^2 + 9*q + 5


5*q^2 + 6*q + 1

In [193]:
# Create a list of matroids
matroids_list = [matroids.Uniform(r, n) for r, n in [(2, 3), (2, 4), (2,5), (3, 4), (4, 5), (5,6), (3, 5), (4, 6)]] + \
    [matroids.Wheel(i) for i in range(3, 6)] + \
    [matroids.Whirl(i) for i in range(3, 5)]

# Create a dataframe to store the results
import pandas as pd

data = {
    'Matroid': matroids_list,
    #'Kazhdan-Lusztig': [m.lattice_of_flats().kazhdan_lusztig_polynomial() for m in matroids_list],
    'invKL': [invKL(m) for m in matroids_list],
    'delform_rhs_qtau': [delform_rhs(m, q_tau, invKL) for m in matroids_list],
    'delform_rhs_tau': [delform_rhs(m, tau, invKL) for m in matroids_list],
    'difference': [x-y for x, y in zip(data['invKL'], data['delform_rhs_tau'])]
}

df = pd.DataFrame(data)
display(df)

,Matroid,invKL,delform_rhs_qtau,delform_rhs_tau,difference
0,"U(2, 3): Matroid of rank 2 on 3 elements with ...",2,1,1,1
1,"U(2, 4): Matroid of rank 2 on 4 elements with ...",3,2,2,1
2,"U(2, 5): Matroid of rank 2 on 5 elements with ...",4,3,3,1
3,"U(3, 4): Matroid of rank 3 on 4 elements with ...",2*q + 3,q + 1,q + 1,q + 2
4,"U(4, 5): Matroid of rank 4 on 5 elements with ...",5*q + 4,3*q + 1,3*q + 1,2*q + 3
5,"U(5, 6): Matroid of rank 5 on 6 elements with ...",5*q^2 + 9*q + 5,5*q^2 + 6*q + 1,5*q^2 + 6*q + 1,3*q + 4
6,"U(3, 5): Matroid of rank 3 on 5 elements with ...",5*q + 6,3*q + 3,3*q + 3,2*q + 3
7,"U(4, 6): Matroid of rank 4 on 6 elements with ...",16*q + 10,9*q + 4,9*q + 4,7*q + 6
8,Wheel(3): Regular matroid of rank 3 on 6 eleme...,q + 6,4,4,q + 2
9,Wheel(4): Regular matroid of rank 4 on 8 eleme...,9*q + 14,5*q + 10,5*q + 10,4*q + 4
